In [1]:
# !pip install dash_bootstrap_components

In [2]:
import dash 
from jupyter_dash import JupyterDash
from dash import html 
from dash import dcc

import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output,State
import dash_bootstrap_components as dbc


In [3]:
df= pd.read_csv('StudentsPerformance.csv')
df['total score']= df['math score']+ df['reading score'] + df['writing score']
df.dtypes

gender                         object
race/ethnicity                 object
parental level of education    object
lunch                          object
test preparation course        object
math score                      int64
reading score                   int64
writing score                   int64
total score                     int64
dtype: object

In [4]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,total score
0,female,group B,bachelor's degree,standard,none,72,72,74,218
1,female,group C,some college,standard,completed,69,90,88,247
2,female,group B,master's degree,standard,none,90,95,93,278
3,male,group A,associate's degree,free/reduced,none,47,57,44,148
4,male,group C,some college,standard,none,76,78,75,229


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
 8   total score                  1000 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 70.4+ KB


In [22]:
df.describe()

,math score,reading score,writing score,total score
count,1000.00000,1000.000000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000,203.312000
std,15.16308,14.600192,15.195657,42.771978
min,0.00000,17.000000,10.000000,27.000000
25%,57.00000,59.000000,57.750000,175.000000
50%,66.00000,70.000000,69.000000,205.000000
75%,77.00000,79.000000,79.000000,233.000000
max,100.00000,100.000000,100.000000,300.000000


In [6]:
df["parental level of education"]=df["parental level of education"].apply(lambda x: x.replace("'s degree",""))
df["parental level of education"].unique()

array(['bachelor', 'some college', 'master', 'associate', 'high school',
       'some high school'], dtype=object)

In [7]:
scores= ['reading score','writing score','math score','total score']
binary=['lunch','gender','test preparation course']
multi=['race/ethnicity','parental level of education']

In [9]:
box1 =px.box(df, y="total score", facet_col="parental level of education",color='gender',facet_col_wrap=2, template='ggplot2',title='Distribution of Total Score WRT Sex and Parents education')
box2=px.box(df,x='total score',y='test preparation course',title='Influence of Test Preparation Course on Total Score',template='ggplot2')
area= px.area(df,x='total score',y=['math score','writing score','reading score'],markers=True, template='ggplot2',title='Relationship between Total Score and other scores')
pie=px.pie(df,names='race/ethnicity',values='total score',title="Groups' Total Score Distribution",template='ggplot2')
P_educ_gender=px.bar(df,x='parental level of education',color='gender',barmode='group',title='Distribution of Parents Education level among Genders')
race_gender=px.bar(df,x='race/ethnicity',color='gender',barmode='group',title='Distribution of Ethnicity Groups among Genders')

In [31]:
app = JupyterDash(external_stylesheets=[dbc.themes.VAPOR])

app.layout = html.Div(
    [
        html.Div([html.H1("Exploratory Data Analysis for Students Performances",style={'textAlign':'center','font-family': 'Serif','font-size':'300%'})
                  ],style={'backgroundColor': '#686868', "border":"3x solid gray"}),
                
        html.Hr(),
        html.Hr(),

        dbc.Row(
            [
                dbc.Col([
                    html.Div(html.H3('Influence of binary attributes on Scores',style={'textAlign':'center','font-family': 'italic'})),
                    html.Div([
                        
                        html.Label('Scores Vs Binary-class Variables',style={'font-size':'115%'}),
                        
                        dcc.Dropdown(
                            id='dropdown_input',
                            options=[{'label': i, 'value': i} for i in scores+binary],
                            placeholder="Select an attribute",
                            value=['reading score','gender'],
                            style ={'color': '#FA8072','backgroundColor':'#20374c','border':'purple'},
                            multi=True),
                        dcc.Graph('dist-binary')
                        ]) ], width=6),
                
                dbc.Col([
                    html.Div(html.H3('Influence of multinomial attributes on Scores',style={'textAlign':'center','font-family': 'italic'})),
                    html.Div([
                        
                        html.Label('Scores Vs Multi-class Variables',style={'font-size':'115%'}),
                        
                        dcc.Dropdown(
                            id='dropdown_input2',
                            options= [{'label':i,'value':i} for i in scores+multi],
                            placeholder="Select an attribute",
                            value=['reading score' , 'parental level of education'],
                            style ={'color': '#FA8072','backgroundColor':'#20374c','border':'#686868'},
                            multi=True),
                        dcc.Graph('dist-multi')
                    ]) ], width=6)
        
            ]),
        
        html.Div([], className="space3", style={'width': '100%', 'height': '50px' , 'display': 'inline-block'}),
        
        html.Div(html.H1("Some other Combinations",style={'textAlign':'center','font-family': 'italic'})),
        html.Hr(),
        
        dbc.Row([
            
            dbc.Row([dbc.Col([ html.Div([dcc.Graph(figure=race_gender)],style={'padding': 10, 'flex': 1, 'backgroundColor': "#20374c"}) ], width=4),
                     dbc.Col([ html.Div([dcc.Graph(figure=P_educ_gender)],style={'padding': 10, 'flex': 1, 'backgroundColor': "#20374c"}) ], width=4),
                     dbc.Col([ html.Div([dcc.Graph(figure=pie)],style={'padding': 10, 'flex': 1,'backgroundColor': "#20374c"}) ], width=4)
                 ],style={'display': 'flex', 'flex-direction': 'row'}),
            
            html.Hr(),
            
            dbc.Row([dbc.Col([html.Div([dcc.Graph(figure=area)],style={'padding': 10, 'flex': 1,'backgroundColor': "#20374c"})],width=6),
                     dbc.Col([html.Div([dcc.Graph(figure=box1)],style={'padding': 10, 'flex': 1, 'backgroundColor': "#20374c"})],width=6)
    
            ])
        ]) 
    ])
        
        
@app.callback(
    Output(component_id='dist-binary',component_property= 'figure'),
    Output(component_id='dist-multi',component_property= 'figure'),
    Input(component_id='dropdown_input',component_property= 'value'),
    Input('dropdown_input2', 'value'))

def update_my_output(input_val1,input_val2):
    
    if len(input_val1)==1:
        fig1= px.histogram(df,x=input_val1[0],template='ggplot2',title='{} Distribution'.format(input_val1[0]))

    elif len(input_val1)==2:
        fig1= px.histogram(df, x=input_val1[0], color=input_val1[1], template='ggplot2', title='Distribution of {} WRT {}'.format(input_val1[0],input_val1[1]))

    elif len(input_val1)==3:
        fig1= px.histogram(df, x=input_val1[0], facet_col=input_val1[1], color=input_val1[2], template='ggplot2', title='Distribution of {} WRT {} , {}'.format(input_val1[0],input_val1[1],input_val1[2]))
    
    elif len(input_val1)>3 or len(input_val1)==0:
        fig1= px.histogram(df,x='total score', template='ggplot2', title='Total Score Distribution')
    
    
    if len(input_val2)==1:
        fig2= px.histogram(df,x=input_val2[0],template='ggplot2',title='{} distribution'.format(input_val2[0]))

    elif len(input_val2)==2:
        fig2= px.histogram(df, x=input_val2[0], facet_col=input_val2[1], facet_col_wrap=2, template='ggplot2', title='Distribution of {} WRT {}'.format(input_val2[0],input_val2[1]))

    elif len(input_val2)==3:
        fig2= px.box(df, y=input_val2[0], facet_col=input_val2[1], color=input_val2[2],facet_col_wrap=2, template='ggplot2', title='Distribution of {} WRT {} , {}'.format(input_val2[0],input_val2[1],input_val2[2]))
        fig2.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

    elif len(input_val2)>3 or len(input_val2)==0:
        fig2= px.histogram(df,x='total score', template='ggplot2', title='Total Score Distribution')
        
    return fig1,fig2

app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [11]:
# dbc.Col([ html.Div([dcc.Graph(figure=area)],style={'padding': 10, 'flex': 1,'display': 'inline-block', 'backgroundColor': "#20374c"}) ], width='auto')

In [12]:
# dbc.Col([ html.Div([dcc.Graph(figure=box2)],style={'padding': 10, 'flex': 1,'display': 'inline-block', 'backgroundColor': "#20374c"}) ],width='auto'),
# dbc.Col([ html.Div([dcc.Graph(figure=P_educ_gender)],style={'padding': 10, 'flex': 1,'display': 'inline-block', 'backgroundColor': "#20374c"}) ], width=2),
# dbc.Col([ html.Div([dcc.Graph(figure=race_gender)],style={'padding': 10, 'flex': 1,'display': 'inline-block', 'backgroundColor': "#20374c"}) ], width=2),

In [13]:
# px.bar(df,x='parental level of education',color='gender',barmode='group')


In [14]:
# px.bar(df,x='race/ethnicity',color='gender',barmode='group')